In [1]:
# Imports
from Code.h5Constructor import h5Constructor

In [2]:
gc = h5Constructor('./Dataset/CIFs/Train/', './Dataset/h5/Train/')

In [6]:
gc.gen_h5s()


Constructing graphs from cif files:


  0%|          | 0/28 [00:00<?, ?it/s]

ReO3_CoO3.cifReO3_FeO3.cif
CdI2_CoO2.cif

ReO3_IrO3.cif
ReO3_MoO3.cifAntiFlourite_Ir2O.cifAntiFlourite_Fe2O.cif
AntiFlourite_Co2O.cif


AntiFlourite_Mo2O.cifFlourite_MoO2.cif



100%|██████████| 28/28 [00:00<00:00, 402.22it/s]


In [4]:
from Code.datasetClass import InOrgMatDatasets

test = InOrgMatDatasets('DatasetTest', root='./Dataset/')

Extracting Dataset/DatasetTest/raw/DatasetTest.zip
Processing...
Done!


In [5]:
test.raw_dir

'Dataset/DatasetTest/raw'